# Pacotes

In [1]:
import os
import sys
print(os.environ.get("SPARK_HOME"))
print(os.environ.get("HADOOP_HOME"))
print(os.environ.get("JAVA_HOME"))
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

C:\Users\pedro\spark-3.5.0-bin-hadoop3
C:\Users\pedro\hadoop3.0
C:\Program Files\Java\jdk1.8.0_202


In [2]:
# Padrões
import os
import glob
import logging
import secrets
import chardet
from functools import reduce

# Pyspark
import pyspark
from pyspark import SparkContext, SQLContext, SparkConf, StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import (
    regexp_replace, when, length, to_date, upper, lower, col, split, explode,
    coalesce, concat_ws, concat, lit, broadcast, regexp_extract, month, year,
    expr, udf, row_number, isnan, count
)

# Geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Pacote de geração de gráficos
import seaborn as sns
import matplotlib.pyplot as plt
import mpld3
import missingno as msno  # Pacote para ver dados missing
import plotly.express as px
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

# Configuração do Spark
import findspark
findspark.init()

# Inicialização do modo offline para Plotly
pyoff.init_notebook_mode(connected=True)

# Remover imports não utilizados, se necessário
# from ReceitaCNPJApi import ReceitaCNPJApi
# from ReceitaLT import ReceitaLT

In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, monotonically_increasing_id, var_samp, when
from pyspark.sql.types import StringType

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import (ChiSqSelector, IndexToString, RFormula, StringIndexer,
                                UnivariateFeatureSelector, VarianceThresholdSelector, VectorAssembler, VectorSlicer)
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

from xgboost.spark import SparkXGBClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel 
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import functions as F

from hyperopt import STATUS_OK, Trials, fmin, hp, space_eval, tpe
from hyperopt.pyll import scope
import mlflow

In [3]:
spark = (SparkSession.builder 
    .master("local[*]") 
    .appName("Spark Optimization")   
    .config("spark.driver.cores", "3")   # Alocando 3 núcleos para o driver
    .config("spark.driver.memory", "12g")  # 12 GB de RAM para o driver
    .config("spark.executor.instances", "5")   # Configurando para 5 executores
    .config("spark.executor.cores", "2")   # Cada executor terá 2 núcleos
    .config("spark.executor.memory", "6g")   # Cada executor terá 6 GB de RAM
    .config("spark.executor.memoryOverhead", "2g")   # Overhead adicional para evitar spill para disco
    .config("spark.memory.fraction", "0.6")  
    .config("spark.memory.storageFraction", "0.5")   
    .config("spark.memory.offHeap.enabled", "true")   
    .config("spark.memory.offHeap.size", "4g")   # Memória off-heap adicional para operações fora do heap JVM
    .config("spark.driver.maxResultSize", "4g")   
    .config("spark.sql.autoBroadcastJoinThreshold", "400m")  # Broadcast join otimizado para joins pequenos
    .config("spark.default.parallelism", "32")   # Paralelismo adequado ao tamanho do dataset
    .config("spark.sql.shuffle.partitions", "32")   # Número de partições para operações de shuffle
    .config("spark.sql.repl.eagerEval.enabled", True)   
    .config("spark.sql.adaptive.enabled", True) 
    .config("spark.sql.cbo.enabled", True) 
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)  
    .config("spark.shuffle.compress", "true")   
    .config("spark.storage.level", "MEMORY_AND_DISK")   
    .config("spark.rdd.compress", "true")   
    .getOrCreate())

In [4]:
spark

# Dataset

In [284]:
df = spark.read.parquet('C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_telefone_tratado_final/').cache()

# Refinamento

In [285]:
df = df.withColumn(
    "renda_per_capita",
    F.when(F.col("renda_per_capita").isNull(), F.col("RENDA_PERCAPTA_MUN"))
    .otherwise(F.col("renda_per_capita"))
)

df = df.drop(df.RENDA_PERCAPTA_MUN,df.TEL1 , df.UF_NOME, df.ENDERECO_COMPLETO, df.DT_INICIO_ATIVIDADE, df.DT_SIT_CADASTRAL, 
             df.NM_SIT_CADASTRAL, df.NM_MOTIVO, df.NM_MOTIVO, df.ano_sit_cadastral, df.mes_sit_cadastral, df.CNPJ_BASICO)

# Especificar as colunas que você deseja transformar
columns_to_cast = [
    "LON",
    "LAT",
    "renda_per_capita",
    "PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000",
    "PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100",
    "IDHM_2010",
    "CAP_SOCIAL"
]

# Substituir ',' por '.' nas colunas selecionadas
for column in columns_to_cast:
    df = df.withColumn(column, F.regexp_replace(F.col(column), ',', '.'))

# Aplicar a transformação de tipo para FLOAT
for column in columns_to_cast:
    df = df.withColumn(column, F.col(column).cast("float"))

In [286]:
df = df.withColumn(
    "OPERADORA",
    F.when(F.col("TIPO_TELEFONE_FINAL") == 'TELEFONE FIXO', F.col("TIPO_TELEFONE_FINAL"))
    .otherwise(F.col("OPERADORA"))
)

df = df.filter(df.NM_QUALIFICACAO == 'Empresário')
df = df.filter(df.NAT_JURICA == 'Empresário (Individual)')
df = df.filter(df.CAP_SOCIAL > 0)
df = df.filter(df.NM_MATRIZ_FILIAL == 'MATRIZ')

In [287]:
df_mun_uf = spark.read.format("csv").option("header", "true").option("delimiter", ",").option("encoding", 'UTF-8').load("C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_mun_cod_lat_long.csv")

# Dicionário de mapeamento de NOME_UF para suas respectivas siglas
uf_mapping = {
    "Rondônia": "RO", "Acre": "AC", "Amazonas": "AM", "Roraima": "RR", "Pará": "PA",
    "Amapá": "AP", "Tocantins": "TO", "Maranhão": "MA", "Piauí": "PI", "Ceará": "CE",
    "Rio Grande do Norte": "RN", "Paraíba": "PB", "Pernambuco": "PE", "Alagoas": "AL",
    "Sergipe": "SE", "Bahia": "BA", "Minas Gerais": "MG", "Espírito Santo": "ES",
    "Rio de Janeiro": "RJ", "São Paulo": "SP", "Paraná": "PR", "Santa Catarina": "SC",
    "Rio Grande do Sul": "RS", "Mato Grosso do Sul": "MS", "Mato Grosso": "MT",
    "Goiás": "GO", "Distrito Federal": "DF"
}

# Aplicar o mapeamento para transformar NOME_UF nas siglas
df_mun_uf = df_mun_uf.replace(uf_mapping, subset=["UF"])

# Remover acentos e aplicar upper no nome dos municípios
from pyspark.sql.functions import col, regexp_replace, upper

# Remover acentos e aplicar upper no nome dos municípios
df_mun_uf = df_mun_uf.withColumn("NOME_MUNICIPIO", upper(regexp_replace(col("NOME_MUNICIPIO"), r'[ÁÀÃÂ]', 'A')))
df_mun_uf = df_mun_uf.withColumn("NOME_MUNICIPIO", upper(regexp_replace(col("NOME_MUNICIPIO"), r'[ÉÈÊ]', 'E')))
df_mun_uf = df_mun_uf.withColumn("NOME_MUNICIPIO", upper(regexp_replace(col("NOME_MUNICIPIO"), r'[ÍÌÎ]', 'I')))
df_mun_uf = df_mun_uf.withColumn("NOME_MUNICIPIO", upper(regexp_replace(col("NOME_MUNICIPIO"), r'[ÓÒÕÔö]', 'O')))
df_mun_uf = df_mun_uf.withColumn("NOME_MUNICIPIO", upper(regexp_replace(col("NOME_MUNICIPIO"), r'[ÚÙÛü]', 'U')))
df_mun_uf = df_mun_uf.withColumn("NOME_MUNICIPIO", upper(regexp_replace(col("NOME_MUNICIPIO"), r'[çÇ]', 'C')))


# Transformar a coluna NOME_MUNICIPIO em caixa alta
df_mun_uf = df_mun_uf.withColumn("NOME_MUNICIPIO", upper(col("NOME_MUNICIPIO")))

df = df.join(df_mun_uf,(df["UF"] == df_mun_uf["UF_SIGLA"]) & (df["MUNICIPIO"] == df_mun_uf["NOME_MUNICIPIO"]), 'left').drop(df_mun_uf.UF_SIGLA,df_mun_uf.UF,df_mun_uf.COD_MUN,df_mun_uf.NOME_MUNICIPIO,df_mun_uf.COD_MUN_COMPL)

df = df.withColumn(
    "LAT",
    F.when(F.col("LAT").isNull(), F.col("LATITUDE"))
    .otherwise(F.col("LAT"))).drop(df.LATITUDE)

df = df.withColumn(
    "LON",
    F.when(F.col("LON").isNull(), F.col("LONGITUDE"))
    .otherwise(F.col("LON"))).drop(df.LONGITUDE)

In [288]:
# Lista de DDDs válidos no Brasil
ddds_validos = [
    '68', '96', '92', '97', '91', '93', '94', '69', '95', '63', '82', '71', '73', '74', '75', '77', '85', '88', '98', '99', 
    '83', '81', '87', '86', '89', '84', '79', '61', '62', '64', '65', '66', '67', '27', '28', '31', '32', '33', '34', '35', 
    '37', '38', '21', '22', '24', '11', '12', '13', '14', '15', '16', '17', '18', '19', '41', '42', '43', '44', '45', '46', 
    '51', '53', '54', '55', '47', '48', '49'
]

# Filtrar os registros em que DDD1 é válido
df = df.filter(df["DDD1"].isin(ddds_validos))

In [268]:
df.printSchema()

root
 |-- CNPJ: string (nullable = true)
 |-- NOME_EMPRESA: string (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- CAP_SOCIAL: float (nullable = true)
 |-- NAT_JURICA: string (nullable = true)
 |-- NM_QUALIFICACAO: string (nullable = true)
 |-- SEXO_PROB: string (nullable = true)
 |-- NM_MATRIZ_FILIAL: string (nullable = true)
 |-- CNAE: string (nullable = true)
 |-- ano_cadastro: integer (nullable = true)
 |-- mes_cadastro: integer (nullable = true)
 |-- TIPO_LOUGRADOURO: string (nullable = true)
 |-- CEP: integer (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- PROVEDOR: string (nullable = true)
 |-- LON: string (nullable = true)
 |-- LAT: string (nullable = true)
 |-- DDD1: string (nullable = true)
 |-- TIPO_TELEFONE_FINAL: string (nullable = true)
 |-- OPERADORA: string (nullable = true)
 |-- renda_per_capita: float (nullable = true)
 |-- PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000: float (nullable = true)
 |--

In [269]:
df

CNPJ,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,NAT_JURICA,NM_QUALIFICACAO,SEXO_PROB,NM_MATRIZ_FILIAL,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010
11630805,LUIZ ANTONIO MELO...,TOCA DO TONINHO,1.0,Empresário (Indiv...,Empresário,M,MATRIZ,Bares e outros es...,2010,3,TRAVESSA,66035180,BELEM,PA,HOTMAIL,-48.48058,-1.450814,91,TELEFONE CELULAR,claro,3789.9766,1.880104E7,13506.19,Demais serviços,Comércio e repara...,0.746
11642995,NAILTON OLIVEIRA ...,M.Z CARTUCHOS E T...,20000.0,Empresário (Indiv...,Empresário,M,MATRIZ,Comércio varejist...,2010,3,RUA,74470620,GOIANIA,GO,HOTMAIL,-49.359314,-16.627825,62,TELEFONE CELULAR,tim,921.5463,2.9038012E7,22304.47,Demais serviços,Comércio e repara...,0.799
11853430,CRISTIANO JUNIO D...,CRISTIANO JUNIO D...,1.0,Empresário (Indiv...,Empresário,M,MATRIZ,Serviços de organ...,2010,4,RUA,55490000,ALTINHO,PE,HOTMAIL,-36.06183,-8.490301,81,TELEFONE CELULAR,tim,452.6179,88401.08,3953.01,"Administração, de...",Demais serviços,0.598
11922321,JOAO SOUZA SILVA ...,J.C.L. TURISMO,1.0,Empresário (Indiv...,Empresário,M,MATRIZ,Transporte escolar,2010,5,RUA,83701040,ARAUCARIA,PR,IBEST,-49.392452,-25.597212,41,TELEFONE CELULAR,tim,1201.9717,8993872.0,75447.51,Indústrias de tra...,Demais serviços,0.74
12082287,JANDERSON CANDIDO...,CANDIDO UTILIDADES,10000.0,Empresário (Indiv...,Empresário,M,MATRIZ,Lojas de variedad...,2010,6,RUA,79092540,CAMPO GRANDE,MS,HOTMAIL,-54.6621,-20.508438,67,TELEFONE CELULAR,claro,615.22003,1.508912E7,19167.99,Demais serviços,Comércio e repara...,0.784
12420483,ADENILSON OLIVEIR...,ANK,1.0,Empresário (Indiv...,Empresário,M,MATRIZ,Serviços de organ...,2010,8,RUA DE PEDESTRE,76916000,PRESIDENTE MEDICI,RO,HOTMAIL,-61.90514,-11.170741,69,TELEFONE CELULAR,vivo,993.1297,235588.86,10555.53,"Administração, de...","Pecuária, inclusi...",0.664
12450256,EMILIANO EMMANUEL...,RIOTURISMO.NET,1.0,Empresário (Indiv...,Empresário,M,MATRIZ,Operadores turíst...,2010,8,RUA,22031100,RIO DE JANEIRO,RJ,HOTMAIL,-43.1922,-22.962667,21,TELEFONE CELULAR,tim,644.26587,2.081536E8,32919.88,Demais serviços,"Administração, de...",0.799
12525458,MARIA APARECIDA R...,MARIA ROSA SEMIJOIAS,1.0,Empresário (Indiv...,Empresário,F,MATRIZ,Cursos preparatór...,2010,9,RUA,79010800,CAMPO GRANDE,MS,HOTMAIL,-54.60077,-20.439486,67,TELEFONE CELULAR,tim,1149.4879,1.508912E7,19167.99,Demais serviços,Comércio e repara...,0.784
12554991,FLAVIO RIBEIRO BO...,DELICIAS DA OLEGARIO,30000.0,Empresário (Indiv...,Empresário,M,MATRIZ,"Lanchonetes, casa...",2010,9,AVENIDA,30180110,BELO HORIZONTE,MG,GMAIL,-43.943817,-19.91843,31,TELEFONE CELULAR,claro,1796.4888,5.9203076E7,24922.95,Demais serviços,Comércio e repara...,0.81
12567779,JONE CLECIO FEITO...,JONE CALHAS E COIFAS,10000.0,Empresário (Indiv...,Empresário,M,MATRIZ,Produção de artef...,2010,9,RUA,58040260,JOAO PESSOA,PB,HOTMAIL,-34.85934,-7.127913,83,TELEFONE CELULAR,vivo,770.8474,1.0843958E7,14987.9,Demais serviços,"Administração, de...",0.763


## Missing

In [242]:
def missing_values_table_spark(df):
    """
    Cria uma tabela resumindo a quantidade e a porcentagem de valores ausentes em cada coluna do DataFrame PySpark.

    Args:
    df (spark.DataFrame): DataFrame para análise de valores ausentes.

    Returns:
    spark.DataFrame: Uma tabela com o número e a porcentagem de valores ausentes por coluna.
    """
    # Calcula o número total de linhas no DataFrame
    total_rows = df.count()

    # Calcula o número total de valores ausentes por coluna
    missing_count = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

    # Calcula a porcentagem de valores ausentes por coluna
    missing_percent = df.select([(count(when(col(c).isNull(), c)) / total_rows * 100).alias(c) for c in df.columns])

    # Preparando para juntar contagens e percentagens
    missing_count = missing_count.withColumnRenamed(missing_count.columns[0], 'Missing Values')
    missing_percent = missing_percent.withColumnRenamed(missing_percent.columns[0], '% of Total Values')

    # Junta as contagens e as percentagens em um DataFrame
    missing_table = missing_count.join(missing_percent)

    # Ordena as colunas com valores ausentes por porcentagem de forma decrescente
    missing_table = missing_table.orderBy(col('% of Total Values').desc())

    # Imprime um resumo das colunas com valores ausentes
    print("Your selected dataframe has " + str(len(df.columns)) + " columns.\n"      
          "There are " + str(missing_table.count()) + " columns that have missing values.")

    return missing_table

In [243]:
missing_values_table_spark(df)

Your selected dataframe has 27 columns.
There are 1 columns that have missing values.


Missing Values,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,NAT_JURICA,NM_QUALIFICACAO,SEXO_PROB,NM_MATRIZ_FILIAL,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010,% of Total Values,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,NAT_JURICA,NM_QUALIFICACAO,SEXO_PROB,NM_MATRIZ_FILIAL,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010
99039,99039,12754856,0,0,0,509392,0,0,99039,0,60,2,0,0,3786777,49504,49504,99039,99039,584664,12524,2387303,2387303,2387303,2387303,136502,0.5259480503162757,0.5259480503162757,67.73484834524632,0.0,0.0,0.0,2.70513362661889,0.0,0.0,0.5259480503162757,0.0,3.186308728781241...,1.062102909593747...,0.0,0.0,20.109734348413408,0.26289171218264434,0.26289171218264434,0.5259480503162757,0.5259480503162757,3.104866677673593,0.06650888419876046,12.677807311909408,12.677807311909408,12.677807311909408,12.677807311909408,0.7248958568268284


## Tratamento missing

In [289]:
# Especificar as colunas para verificar missing values
columns_to_check = ["NOME_EMPRESA", "CNAE", "ano_cadastro", "DDD1", "IDHM_2010", 'CEP','renda_per_capita','LON','LAT']

# Remover as linhas com valores missing nas colunas especificadas
df = df.dropna(subset=columns_to_check)

df = df.fillna({
    "TIPO_LOUGRADOURO": "SI",
    "MUNICIPIO": "SI",
    "UF": "SI",
    "TIPO_TELEFONE_FINAL": "SI",
    "OPERADORA": "SI",
    'PROVEDOR': 'NAO_CADASTRADO',
    "ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO": "SI",
    "ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO":  "SI",
    'PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000': 0,
    'PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100': 0,
    "SEXO_PROB": "SI",
    "NOME_FANTASIA": "NAO_INFORMADO"})

In [290]:
    dic_provedor = {'OULOOK': 'OUTLOOK','AUTLOOK': 'OUTLOOK','HOMAIL': 'HOTMAIL','HOTMAI': 'HOTMAIL',
                    'HOTAMIL': 'HOTMAIL','HOTMIL': 'HOTMAIL','HOTMIAL': 'HOTMAIL','GMAILL': 'GMAIL', 'GEMAIL': 'GMAIL', 
        'OUTLOOK': 'OUTLOOK', 'OUTIOOK': 'OUTLOOK', 'OUTLLOK': 'OUTLOOK', 'OUTLLOOK': 'OUTLOOK', 
        'OUTLOCK': 'OUTLOOK', 'OUTLOK': 'OUTLOOK', 'OUTLOKK': 'OUTLOOK', 'OUTLOOCK': 'OUTLOOK', 'OUTLOOK': 'OUTLOOK', 
        'OUTLOOKL': 'OUTLOOK', 'OUTLOOL': 'OUTLOOK', 'OUTLOOOK': 'OUTLOOK', 'OUTLUK': 'OUTLOOK', 'OUTOLOOK': 'OUTLOOK',
        'OUTOOK': 'OUTLOOK', 'OUTOOLK': 'OUTLOOK', 'OUTTLOOK': 'OUTLOOK', 'OUTULOOK': 'OUTLOOK', 'POP': 'POP',
        'PROTON': 'PROTONMAIL', 'PROTONMAIL': 'PROTONMAIL', 'PUTLOOK': 'OUTLOOK', 'R7': 'R7', 'ROCKETMAIL': 'ROCKETMAIL', 
        'ROCKTMAIL': 'ROCKETMAIL', 'ROTMAIL': 'HOTMAIL', 'SERCOMTEL': 'SERCOMTEL', 'SETELAGOASGML': 'GMAIL', 
        'SUPERIG': 'SUPERIG', 'TAHOO': 'YAHOO', 'TERRA': 'TERRA', 'TERRRA': 'TERRA', 'TMAIL': 'GMAIL', 
        'TVGLOBO': 'GLOBO', 'UAHOO': 'YAHOO', 'UAI': 'UAI', 'UFV': 'UFV', 'UNESP': 'UNESP', 'UNOCHAPECO': 'UNOCHAPECO', 
        'UO': 'UOL', 'UOL': 'UOL', 'UOTLOOK': 'OUTLOOK', 'UPF': 'UPF', 'USP': 'USP', 'UTLOOK': 'OUTLOOK', 'VELOXMAIL': 'VELOXMAIL',
        'WINDOWSLIVE': 'WINDOWSLIVE', 'YAAHOO': 'YAHOO', 'YAGOO': 'YAHOO', 'YAHAOO': 'YAHOO', 'YAHHO': 'YAHOO', 'YAHHOO': 'YAHOO', 
        'YAHO': 'YAHOO', 'YAHOO': 'YAHOO', 'YAHOOCOM': 'YAHOO', 'YAHOOL': 'YAHOO', 'YAHOOO': 'YAHOO', 'YAHOOU': 'YAHOO', 
        'YANHOO': 'YAHOO', 'YAOO': 'YAHOO', 'YAOOL': 'YAHOO', 'YAROO': 'YAHOO', 'YHAOO': 'YAHOO', 'YHOO': 'YAHOO', 'YMAIL': 'YMAIL', 
        'YOHOO': 'YAHOO', 'YOPMAIL': 'HOTMAIL', 'ZIPMAIL': 'ZIPMAIL', '_HOTMAIL': 'HOTMAIL', 'GMAUL': 'GMAIL','GMALE': 'GMAIL', 
        'GMAILE': 'GMAIL', 'GMILE': 'GMAIL', 'HOTMEL': 'HOTMAIL', 'HOTMELL': 'HOTMAIL','HOTMEAL': 'HOTMAIL', 'OUTLOKES': 'OUTLOOK', 
        'OTLOOKS': 'OUTLOOK', 'YAHU': 'YAHOO', 'YOHU': 'YAHOO', 'YAHUO': 'YAHOO', 'YAHEE': 'YAHOO', 'UOLL': 'UOL',
        'UOOL': 'UOL', 'UULL': 'UOL', 'ICLODUE': 'ICLOUD', 'ICLAWD': 'ICLOUD', 'ROCKEDMAIL': 'ROCKETMAIL', 'ROKETMAIL': 'ROCKETMAIL',
        'OUTLOKE': 'OUTLOOK', 'OUTLOOCKE': 'OUTLOOK', 'YAAHO': 'YAHOO', 'YAHOOE': 'YAHOO', 'YAHUE': 'YAHOO', 'HOTMILE': 'HOTMAIL', 'HOTMELE': 'HOTMAIL',
        'FACEBOKE': 'FACEBOOK', 'FACBOOK': 'FACEBOOK', 'FCEBOOK': 'FACEBOOK', 'BOLL': 'BOL', 'BOLLE': 'BOL', 'BULE': 'BOL', 'GLOBOE': 'GLOBO',
        'GLOBU': 'GLOBO', 'GMILE': 'GMAIL', 'MSNE': 'MSN', 'MSNN': 'MSN', 'ICLOOUD': 'ICLOUD', 'OUTLUKE': 'OUTLOOK', 'OUTLLOKE': 'OUTLOOK',
        'PROTONMIAL': 'PROTONMAIL', 'PROTONMALE': 'PROTONMAIL', 'PROTOMAIL': 'PROTONMAIL',  'OTLOOK': 'AUTLOOK',  'OUTLOOK': 'OUTLOOK', 
        'HOTMAL': 'HOTMAIL','HORMAIL': 'HOTMAIL','HOTMALIL': 'HOTMAIL','': 'HOTMAIL','HOTAMAIL': 'HOTMAIL','HOTAIL': 'HOTMAIL',
        'OULOOKCOM': 'OUTLOOK', 'YAHCOM': 'YAHOO',  'YAHOCOM': 'YAHOO','GAMILCOM': 'GMAIL', 'GMALCOM': 'GMAIL',  'HTOMAIL': 'HOTMAIL',
        'HOTMALCOM': 'HOTMAIL',  'HOTMILCOM': 'HOTMAIL', 'HOTMELCOM': 'HOTMAIL', 'ROCKMAIL': 'ROCKETMAIL', 
        'ROKMAIL': 'ROCKETMAIL', 'TERA': 'TERRA', 'TEERA': 'TERRA', 'GAMIL': 'GMAIL',  'GMAI': 'GMAIL',  
        'FACBOOKCOM': 'FACEBOOK', 'FACEBOOKCOM': 'FACEBOOK', 'ICLOWD': 'ICLOUD', 'ICLOUND': 'ICLOUD', 
        'UOOLCOM': 'UOL', 'UOLLCOM': 'UOL', 'UOLCOMBR': 'UOL','LIVECOM': 'LIVE', 'LIVECOMBR': 'LIVE', 
        'GMAICOM': 'GMAIL',  'GMAILCOMBR': 'GMAIL',  'YAHOOBR': 'YAHOO', 'GNAIL': 'GMAIL','GMAL': 'GMAIL',  'GMIL': 'GMAIL',  'GMIAL': 'GMAIL',  'GMEIL': 'GMAIL',  
        'YAHOOOCOMBR': 'YAHOO', 'YAHOOOCOM': 'YAHOO', 'ZIPMAILE': 'ZIPMAIL', 'ZIPMAILL': 'ZIPMAIL',  
        'IBESTT': 'IBEST', 'IBESTE': 'IBEST','GMAIIL': 'GMAIL', 'GMAILLL': 'GMAIL', 'GMIAIL': 'GMAIL', 'GAMAIL': 'GMAIL',
        'HOMTAIL': 'HOTMAIL', 'HOTMALIL': 'HOTMAIL', 'HTMAIL': 'HOTMAIL', 'HOTIMAIL': 'HOTMAIL',
        'YHOO': 'YAHOO', 'YAOO': 'YAHOO', 'YHO': 'YAHOO', 'YHAOO': 'YAHOO', 'OUL': 'UOL', 'UOOL': 'UOL', 'UL': 'UOL','ICLOUND': 'ICLOUD', 'ICLOOWD': 'ICLOUD', 'ICLOUDE': 'ICLOUD',
        'LVE': 'LIVE', 'LIVVE': 'LIVE', 'PROTOMAL': 'PROTONMAIL', 'PROTOMAILL': 'PROTONMAIL', 'BLL': 'BOL', 'BOLE': 'BOL','MNN': 'MSN', 'MN': 'MSN',
        'FACBOOKE': 'FACEBOOK', 'FACEBOK': 'FACEBOOK',    'TERAA': 'TERRA', 'TERR': 'TERRA', 'ÝHOTMAIL': 'HOTMAIL'}


# Aplicar correção nos provedores de e-mails
df = df.withColumn("PROVEDOR", upper(col("PROVEDOR")))  # Primeiro, garantir que está em maiúsculas
df = df.replace(dic_provedor, subset=["PROVEDOR"])

# Lista de provedores válidos
provedores_validos = [
    'GMAIL', 'HOTMAIL', 'NAO_CADASTRADO', 'YAHOO', 'OUTLOOK', 'BOL', 'IG', 'LIVE',
    'UOL', 'TERRA', 'ICLOUD', 'OI', 'GLOBO', 'MSN', 'YMAIL', 'R7',
    'IBEST', 'BRTURBO', 'GLOBOMAIL', 'NETSITE'
]

# Aplicar tratamento: manter provedores válidos, tudo que não for válido será "OUTROS"
df = df.withColumn(
    "PROVEDOR",
    F.when(F.col("PROVEDOR").isin(provedores_validos), F.col("PROVEDOR"))
    .otherwise("OUTROS"))

In [291]:
df = df.withColumn(
    "TIPO_LOUGRADOURO",
    F.when(F.col("TIPO_LOUGRADOURO").rlike(r"^\d{1,2}A RUA$"), "RUA")
     .when(F.col("TIPO_LOUGRADOURO").rlike(r"^\d{1,2}A AVENIDA$"), "AVENIDA")
     .when(F.col("TIPO_LOUGRADOURO").rlike(r"^\d{1,2}A TRAVESSA$"), "TRAVESSA")
     .when(F.col("TIPO_LOUGRADOURO") == "RUA PRINCIPAL", "RUA")
     .when(F.col("TIPO_LOUGRADOURO") == "RUA DE PEDESTRE", "RUA")
     .when(F.col("TIPO_LOUGRADOURO") == "PRAÇA", "PRACA")
     .when(F.col("TIPO_LOUGRADOURO") == "RUA DE LIGACAO", "RUA")
     .when(F.col("TIPO_LOUGRADOURO") == "1A VILA", "VILA")
     .otherwise(F.regexp_replace(F.col("TIPO_LOUGRADOURO"), r"PRA�A", "PRACA"))
)

# Lista de provedores válidos
LOUGRADOUROS_VALIDOS = ['RUA', 'AVENIDA', 'TRAVESSA', 'ESTRADA', 'QUADRA', 'RODOVIA',
                        'PRACA', 'ALAMEDA', 'SITIO', 'PASSAGEM', 'AREA', 'VILA', 'CONJUNTO', 'SERVIDAO', 'FAZENDA',
                        'LOTEAMENTO', 'CAMINHO', 'BECO', 'RESIDENCIAL', 'SETOR', 'COMUNIDADE', 'CONDOMINIO', 'DISTRITO', 'VIA', 'ACESSO', 'CHACARA', 'ESTRADA MUNICIPAL',
                        'LADEIRA', 'VIELA', 'CORREGO', 'ACESSO LOCAL', 'CONJUNTO RESIDENCIAL', 'LARGO', 'COLONIA', 'CAIS', 'PRAIA', 'NUCLEO RURAL', 'ESTRADA VELHA', 'BLOCO',
                        'ENTRADA', 'RAMAL', 'AVENIDA PERIMETRAL', 'RUA PROJETADA', 'JARDIM', 'CONJUNTO HABITACIONA', 'AREA ESPECIAL', 'COND. RESIDENCIAL', 'PARQUE', 'CONDOMINIO RESIDENCI', 'CAMPO']


# Aplicar tratamento: manter provedores válidos, tudo que não for válido será "OUTROS"
df = df.withColumn(
    "TIPO_LOUGRADOURO",
    F.when(F.col("TIPO_LOUGRADOURO").isin(LOUGRADOUROS_VALIDOS), F.col("TIPO_LOUGRADOURO"))
    .otherwise("OUTROS"))

In [246]:
missing_values_table_spark(df)

Your selected dataframe has 27 columns.
There are 1 columns that have missing values.


Missing Values,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,NAT_JURICA,NM_QUALIFICACAO,SEXO_PROB,NM_MATRIZ_FILIAL,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010,% of Total Values,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,NAT_JURICA,NM_QUALIFICACAO,SEXO_PROB,NM_MATRIZ_FILIAL,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [273]:
df.count()

18586107

In [280]:
df.count()

18568511

# Análise do dataset

In [125]:
df.describe()

summary,CNPJ,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,NAT_JURICA,NM_QUALIFICACAO,SEXO_PROB,NM_MATRIZ_FILIAL,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010
count,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,16280789,18573942,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272
mean,3.1092018035522323E7,NULL,NaN,36407.73719543715,NULL,NULL,NULL,NULL,NULL,2017.4444724804114,6.346332658748147,NULL,4.462115303479118E7,NULL,NULL,Infinity,-45.967436210123,-19.165823722080432,42.41075489827626,NULL,NULL,939.8122709205612,7.965226348170106E7,32933.290163712045,NULL,NULL,0.747150483969427
stddev,1.0827338019598354E7,NULL,NaN,8453368.812449822,NULL,NULL,NULL,NULL,NULL,3.5641681796259475,3.3376140678560087,NULL,2.9465555094170175E7,NULL,NULL,NaN,5.465537680775317,7.266357284432423,26.894811715966018,NULL,NULL,661.2156697123469,1.859986435883497E8,29344.201379146452,NULL,NULL,0.06039578460705528
min,00166830,ERLY BATISTA GON...,\tILZETE SANTOS S...,0.0,Associação Privada,Administrador,F,FILIAL,Abate de aves,2009,1,10A AVENIDA,0,ABADIA DE GOIAS,AC,,-32.412697,-0.001461,01,TELEFONE CELULAR,SI,0.0,-19046.434,-1459.83,"Administração, de...","Administração, de...",0.418
max,98014649,ZYR RUBIA LISBOA ...,{ MAD },3.21016443E10,Sociedade em Cont...,Titular de Empres...,SI,MATRIZ,atividades de des...,2024,12,�REA ESPECIAL,99990000,ZORTEA,TO,ÝHOTMAIL,-72.915764,6.1E-5,99,TELEFONE FIXO,vivo,48093.81,8.2898061E8,920834.0,SI,SI,0.862


In [126]:
df.summary()

summary,CNPJ,NOME_EMPRESA,NOME_FANTASIA,CAP_SOCIAL,NAT_JURICA,NM_QUALIFICACAO,SEXO_PROB,NM_MATRIZ_FILIAL,CNAE,ano_cadastro,mes_cadastro,TIPO_LOUGRADOURO,CEP,MUNICIPIO,UF,PROVEDOR,LON,LAT,DDD1,TIPO_TELEFONE_FINAL,OPERADORA,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,IDHM_2010
count,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,16280789,18573942,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272,18583272
mean,3.1092018035522323E7,NULL,NaN,36407.73719543715,NULL,NULL,NULL,NULL,NULL,2017.4444724804114,6.346332658748147,NULL,4.462115303479118E7,NULL,NULL,Infinity,-45.967436210123,-19.165823722080432,42.41075489827626,NULL,NULL,939.8122709205612,7.965226348170106E7,32933.290163712045,NULL,NULL,0.747150483969427
stddev,1.0827338019598354E7,NULL,NaN,8453368.812449822,NULL,NULL,NULL,NULL,NULL,3.5641681796259475,3.3376140678560087,NULL,2.9465555094170175E7,NULL,NULL,NaN,5.465537680775317,7.266357284432423,26.894811715966018,NULL,NULL,661.2156697123469,1.859986435883497E8,29344.201379146452,NULL,NULL,0.06039578460705528
min,00166830,ERLY BATISTA GON...,\tILZETE SANTOS S...,0.0,Associação Privada,Administrador,F,FILIAL,Abate de aves,2009,1,10A AVENIDA,0,ABADIA DE GOIAS,AC,,-32.412697,-0.001461,01,TELEFONE CELULAR,SI,0.0,-19046.434,-1459.83,"Administração, de...","Administração, de...",0.418
25%,2.1954596E7,NULL,96.0,500.0,NULL,NULL,NULL,NULL,NULL,2015,3,NULL,18022150,NULL,NULL,32.0,-49.14603,-23.535249500000003,19.0,NULL,NULL,565.28,596941.3,15519.13,NULL,NULL,0.717
50%,3.1345719E7,NULL,135135.0,1500.0,NULL,NULL,NULL,NULL,NULL,2018,6,NULL,38230000,NULL,NULL,163.0,-46.56479,-22.229084,35.0,NULL,NULL,784.5557,6204124.5,28441.08,NULL,NULL,0.758
75%,4.0606685E7,NULL,5.224256909E9,5000.0,NULL,NULL,NULL,NULL,NULL,2021,9,NULL,72236800,NULL,NULL,2010.0,-43.18105,-15.76034,65.0,NULL,NULL,1109.01,3.9566468E7,47240.71,NULL,NULL,0.799
max,98014649,ZYR RUBIA LISBOA ...,{ MAD },3.21016443E10,Sociedade em Cont...,Titular de Empres...,SI,MATRIZ,atividades de des...,2024,12,�REA ESPECIAL,99990000,ZORTEA,TO,ÝHOTMAIL,-72.915764,6.1E-5,99,TELEFONE FIXO,vivo,48093.81,8.2898061E8,920834.0,SI,SI,0.862


In [258]:
df_count1 = df.groupBy('TIPO_LOUGRADOURO').count().orderBy(col('count').desc()).toPandas()
df_count1.head(10)

,TIPO_LOUGRADOURO,count
0,RUA,13974719
1,AVENIDA,2543532
2,TRAVESSA,382086
3,ESTRADA,349594
4,QUADRA,264808
5,RODOVIA,142251
6,PRACA,134975
7,ALAMEDA,102944
8,OUTROS,79060
9,SITIO,53686


In [249]:
df_count2 = df.groupBy('ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO').count().orderBy(col('count').desc()).toPandas()
df_count2

,ATIVIDADE_COM_SEGUNDO_MAIOR_VALOR_ADICIONADO_BRUTO,count
0,Comércio e reparação de veículos automotores e...,4751152
1,"Administração, defesa, educação e saúde públic...",4668689
2,Demais serviços,3447741
3,Indústrias de transformação,2708827
4,SI,2265636
5,"Agricultura, inclusive apoio à agricultura e a...",347015
6,Indústrias extrativas,128823
7,Construção,106337
8,"Pecuária, inclusive apoio à pecuária",90505
9,"Eletricidade e gás, água, esgoto, atividades d...",49470


In [250]:
df_count3 = df.groupBy('SEXO_PROB').count().orderBy(col('count').desc()).toPandas()
df_count3

,SEXO_PROB,count
0,M,9434464
1,F,8648516
2,SI,503127


In [251]:
df_count4 = df.groupBy('NM_QUALIFICACAO').count().orderBy(col('count').desc()).toPandas()
df_count4

,NM_QUALIFICACAO,count
0,Empresário,18586107


In [252]:
df_count5 = df.groupBy('NAT_JURICA').count().orderBy(col('count').desc()).toPandas()
df_count5

,NAT_JURICA,count
0,Empresário (Individual),18586107


In [278]:
df_count6 = df.groupBy('DDD1').count().orderBy(col('count').desc()).toPandas()
df_count6

,DDD1,count
0,11,2726012
1,21,1587131
2,31,915232
3,51,618166
4,19,597299
...,...,...
62,68,42884
63,96,37533
64,89,35351
65,95,32265


In [254]:
df_count7 = df.groupBy('OPERADORA').count().orderBy(col('count').desc()).toPandas()
df_count7

,OPERADORA,count
0,TELEFONE FIXO,5938523
1,vivo,4750221
2,tim,3870723
3,claro,3444204
4,SI,482724
5,algar,90995
6,oi,5214
7,sercomtel,3418
8,datora,51
9,america_net,27


In [255]:
df_count8 = df.groupBy('TIPO_TELEFONE_FINAL').count().orderBy(col('count').desc()).toPandas()
df_count8

,TIPO_TELEFONE_FINAL,count
0,TELEFONE CELULAR,12647584
1,TELEFONE FIXO,5938523


In [256]:
df_count9 = df.groupBy('PROVEDOR').count().orderBy(col('count').desc()).toPandas()
df_count9

,PROVEDOR,count
0,GMAIL,7907798
1,HOTMAIL,4328099
2,NAO_CADASTRADO,3721728
3,YAHOO,753869
4,OUTROS,617198
5,OUTLOOK,477134
6,BOL,218190
7,IG,100583
8,LIVE,99759
9,UOL,88354


In [259]:
df_count10 = df.groupBy('ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO').count().orderBy(col('count').desc()).toPandas()
df_count10

,ATIVIDADE_COM_MAIOR_VALOR_ADICIONADO_BRUTO,count
0,Demais serviços,12358378
1,SI,2265636
2,"Administração, defesa, educação e saúde públic...",2026058
3,Indústrias de transformação,964715
4,"Agricultura, inclusive apoio à agricultura e a...",391950
5,Indústrias extrativas,263814
6,Comércio e reparação de veículos automotores e...,127069
7,"Eletricidade e gás, água, esgoto, atividades d...",110054
8,"Pecuária, inclusive apoio à pecuária",49765
9,"Produção florestal, pesca e aquicultura",14567


In [281]:
df_count11 = df.groupBy('NM_MATRIZ_FILIAL').count().orderBy(col('count').desc()).toPandas()
df_count11

,NM_MATRIZ_FILIAL,count
0,MATRIZ,18567802
1,FILIAL,709


In [292]:
df_count11 = df.groupBy('NM_MATRIZ_FILIAL').count().orderBy(col('count').desc()).toPandas()
df_count11

,NM_MATRIZ_FILIAL,count
0,MATRIZ,18567802


In [293]:
df.write.mode("overwrite").parquet('C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_final_filtrado_s_missing/')

# Feature Selection

In [64]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, ChiSqSelector, VarianceThresholdSelector, UnivariateFeatureSelector, RFormula, VectorSlicer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
import pandas as pd

def clean_up_columns(df, response_col):
    if "label" in df.columns:
        df = df.drop("label")
    if f"{response_col}_index" in df.columns:
        df = df.drop(f"{response_col}_index")
    if f"old_{response_col}_index" in df.columns:
        df = df.drop(f"old_{response_col}_index")
    return df

def pearson_correlation(df, numeric_features):
    corr_matrix = []
    for x in numeric_features:
        temp = []
        for y in numeric_features:
            temp.append(df.stat.corr(x, y, method='pearson'))
        corr_matrix.append(temp)
    corr_df = pd.DataFrame(corr_matrix, index=numeric_features, columns=numeric_features)
    
    corr_df.to_csv("pearson_correlation.csv", index=False)
    return corr_df

def chi_square_test(df, categorical_cols, response_col):
    df = clean_up_columns(df, response_col)
    
    # Identificar as colunas categóricas
    categorical_cols = [t[0] for t in df.dtypes if t[1] in ('string')]

    # Supondo que uma das colunas numéricas seja a resposta e precisamos transformá-la
    response_col = 'STATUS'  # Variável a ser predita

    # Removendo a variável preditiva
    categorical_cols.remove(f'{response_col}')

    # Indexar colunas categóricas (sem incluir a coluna de resposta)
    indexers = [StringIndexer(inputCol=col, outputCol=col + "_index").setHandleInvalid("keep") for col in categorical_cols]

    # Assemblar todos os índices em um único vetor chamado "features"
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCol="features")

    # Indexar a coluna de resposta separadamente se necessário
    response_indexer = StringIndexer(inputCol=response_col, outputCol=response_col + "_index").setHandleInvalid("keep")

    # Pipeline para transformar os dados, adicionando o indexer da resposta separadamente
    pipeline = Pipeline(stages=indexers + [assembler, response_indexer])
    df_transformed = pipeline.fit(df).transform(df)

    indexed_cols = [col + "_index" for col in categorical_cols]

    # Realizar o teste de Chi-quadrado usando o DataFrame transformado
    chi_square_results = ChiSquareTest.test(df_transformed, "features", "STATUS_index").head()

    # Obter os p-values
    pValues = chi_square_results.pValues

    # Nível de significância
    alpha = 0.05

    # Filtrar colunas significativas baseado nos p-values
    colunas_significativas = [indexed_cols[i] for i, p_val in enumerate(pValues) if p_val < alpha]

    print("Colunas significativas de acordo com o teste de Chi-quadrado:")
    print(colunas_significativas)
    
    return colunas_significativas

def feature_importance_rf(df, categorical_cols, numeric_features, response_col):
    df = clean_up_columns(df, response_col)
    
    # Lista de colunas numéricas
    numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]

    # Identificar as colunas categóricas
    categorical_cols = [t[0] for t in df.dtypes if t[1] in ('string')]

    # Supondo que uma das colunas numéricas seja a resposta e precisamos transformá-la
    response_col = 'STATUS'  # Variável a ser predita

    # Removendo a variável preditiva de categorias, se ela estiver presente
    if response_col in categorical_cols:
        categorical_cols.remove(response_col)

    # Removendo a variável preditiva de numéricas, se ela estiver presente
    if response_col in numeric_features:
        numeric_features.remove(response_col)

    # Indexação de colunas categóricas
    indexers = [
        StringIndexer(inputCol=col, outputCol=col + "_index").setHandleInvalid("keep")
        for col in categorical_cols  # assumindo que estas são as colunas categóricas
    ]

    # Indexação de colunas categóricas e coluna de resposta
    indexers += [StringIndexer(inputCol=response_col, outputCol=response_col + "_index").setHandleInvalid("keep")]

    # Assembler para criar o vetor de features
    assembler = VectorAssembler(
        inputCols=[col + "_index" for col in categorical_cols] + 
        numeric_features,  # adicionando  colunas numéricas
        outputCol="features")

    # Pipeline para processar tudo
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Treinando um RandomForest para avaliar a importância das features
    rf = RandomForestClassifier(featuresCol="features", labelCol=response_col + "_index")
    rf_model = rf.fit(df_transformed)

    # Obtendo importâncias das features
    importances = rf_model.featureImportances

    # Vinculando importâncias com nomes de colunas
    feature_list = [col + "_index" for col in categorical_cols] + numeric_features
    important_features = sorted(zip(feature_list, importances), key=lambda x: x[1], reverse=True)

    # Criando uma lista com as colunas mais significativas baseadas em um threshold de importância
    threshold = 0.05  # Define um limiar de importância, ajuste conforme necessário
    significant_columns = [name for name, importance in important_features if importance > threshold]

    print("Colunas significativas com base na importância do RandomForest:")
    print(significant_columns)
    
    return significant_columns

def chi_sq_selector(df, categorical_cols, response_col, numTopFeatures=4):
    df = clean_up_columns(df, response_col)
    
    # Lista de colunas categóricas
    categorical_cols = [t[0] for t in df.dtypes if t[1] == 'string']

    # Supondo que uma das colunas numéricas seja a resposta e precisamos transformá-la
    response_col = 'STATUS'  # Variável a ser predita, que deve ser transformada em numérica

    # Indexação de colunas categóricas e coluna de resposta
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_index") for c in categorical_cols] + \
               [StringIndexer(inputCol=response_col, outputCol="label")]

    # Assembler para combinar as colunas categóricas indexadas em um vetor de features
    assembler = VectorAssembler(
        inputCols=[c+"_index" for c in categorical_cols],
        outputCol="features")

    # Configuração do ChiSqSelector
    selector = ChiSqSelector(numTopFeatures=numTopFeatures, featuresCol="features",
                             outputCol="selectedFeatures", labelCol="label")

    # Pipeline para processar tudo
    pipeline = Pipeline(stages=indexers + [assembler, selector])
    model = pipeline.fit(df)
    df_transformed = model.transform(df)

    # Obter o modelo do selector para acessar os índices das features selecionadas
    selected_features_model = model.stages[-1]
    selected_indices = selected_features_model.selectedFeatures

    # Criar uma lista de nomes de colunas mapeando de volta usando os índices selecionados
    selected_feature_names = [categorical_cols[index] for index in selected_indices]

    # Exibir os nomes das colunas selecionadas
    print("Colunas selecionadas pelo teste Chi-quadrado:")
    print(selected_feature_names)

    return selected_feature_names

def variance_threshold_selector(df, numeric_features, varianceThreshold=0.5):
    df = clean_up_columns(df, response_col)
        
    # Lista de colunas numéricas
    numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]

    # Assembler para criar o vetor de features numéricas
    assembler = VectorAssembler(
        inputCols=numeric_features,
        outputCol="features")

    # Aplicar o assembler ao DataFrame
    df_assembled = assembler.transform(df)

    # Configurando o VarianceThresholdSelector
    selector = VarianceThresholdSelector(
        featuresCol="features",
        outputCol="selectedFeatures",
        varianceThreshold=varianceThreshold  # Definir um limiar de variância apropriado
    )

    # Aplicando o selector ao DataFrame
    model = selector.fit(df_assembled)
    df_selected = model.transform(df_assembled)

    # Obter os índices das features selecionadas
    selected_indices = model.selectedFeatures

    # Criar uma lista de nomes de colunas mapeando de volta usando os índices selecionados
    selected_feature_names = [numeric_features[index] for index in selected_indices]

    # Exibir os nomes das colunas selecionadas
    print("Colunas selecionadas com variância acima do limiar:", selected_feature_names)

    return selected_feature_names

def univariate_feature_selector(df, categorical_cols, numeric_features, response_col, selectionThreshold=5):
    df = clean_up_columns(df, response_col)
    
    # Lista de colunas numéricas
    numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]

    # Identificar as colunas categóricas
    categorical_cols = [t[0] for t in df.dtypes if t[1] in ('string')]

    # Supondo que uma das colunas numéricas seja a resposta e precisamos transformá-la
    response_col = 'STATUS'  # Variável a ser predita

    # Indexação de colunas categóricas e coluna de resposta
    indexers = [StringIndexer(inputCol=c, outputCol=c+"_index", handleInvalid="keep") for c in categorical_cols]
    indexer_response = StringIndexer(inputCol="STATUS", outputCol="label", handleInvalid="keep")

    # Assembler para combinar colunas numéricas e categóricas indexadas em um vetor de features
    assembler = VectorAssembler(
        inputCols=[c+"_index" for c in categorical_cols] + numeric_features,
        outputCol="features")

    selector = UnivariateFeatureSelector(
        featuresCol="features",
        outputCol="selectedFeatures",
        labelCol="label",  # Usar "label", pois é o nome configurado pelo indexer_response
        selectionMode="numTopFeatures"  # ou use outro modo conforme necessário
    )

    selector.setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(selectionThreshold)  # ajuste conforme necessário

    # Adicionando a transformação da label ao pipeline
    pipeline = Pipeline(stages=indexers + [indexer_response, assembler, selector])
    model = pipeline.fit(df)
    df_transformed = model.transform(df)

    # Obter os índices das features selecionadas
    selected_indices = model.stages[-1].selectedFeatures  # O último estágio é o UnivariateFeatureSelector

    # Lista de todas as colunas que foram usadas para criar o vetor de features
    all_features = [col + "_index" for col in categorical_cols] + numeric_features

    # Mapear os índices selecionados de volta para os nomes das colunas
    selected_feature_names = [all_features[index] for index in selected_indices]

    # Imprimir os nomes das colunas das features selecionadas
    print("Colunas selecionadas:", selected_feature_names)

    return selected_feature_names

def r_formula(df, categorical_cols, numeric_features, response_col):
    # Definindo a fórmula
    predictor_cols = categorical_cols + numeric_features
    formula_expression = f"{response_col} ~ " + " + ".join(predictor_cols)

    # Configuração do RFormula
    formula = RFormula(
        formula=formula_expression,
        featuresCol="features",
        labelCol="label"
    )

    # Pipeline para aplicar a RFormula
    pipeline = Pipeline(stages=[formula])
    df_transformed = pipeline.fit(df).transform(df)

    # Visualizando os resultados
    df_transformed.select("features", "label").show()

    return df_transformed

def vector_slicer_function(df):
    # Suponha que 'STATUS' seja a variável de resposta e já foi removida das listas se necessário
    response_col = 'STATUS'
    
    df = clean_up_columns(df, response_col)

    # Lista de colunas numéricas e categóricas
    numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]
    categorical_cols = [t[0] for t in df.dtypes if t[1] == 'string']

    # Indexação de colunas categóricas
    indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep") for col in categorical_cols]

    # Assembler para combinar todas as colunas indexadas e numéricas em um vetor de features
    assembler = VectorAssembler(
        inputCols=[col + "_index" for col in categorical_cols] + numeric_features,
        outputCol="features"
    )

    # Pipeline para processar a indexação e assembler
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)

    # Configuração do VectorSlicer para extrair features específicas
    indices_to_slice = [0, 1, 3, 4, 5]  # Os índices que você deseja extrair
    vs = VectorSlicer().setInputCol("features").setOutputCol("selectedFeatures").setIndices(indices_to_slice)
    df_sliced = vs.transform(df_transformed)

    # Mapeando índices de volta aos nomes das colunas
    input_feature_names = [col + "_index" for col in categorical_cols] + numeric_features
    selected_feature_names = [input_feature_names[index] for index in indices_to_slice]

    # Visualizando as features selecionadas e seus nomes
    df_sliced.select("selectedFeatures").show(truncate=False)
    print("Nomes das colunas selecionadas:", selected_feature_names)

    return selected_feature_names


# Criação de um index único

In [21]:
df = df.withColumn("index", monotonically_increasing_id())

# Execução

In [35]:
df.printSchema()

root
 |-- CNPJ: string (nullable = true)
 |-- NOME_EMPRESA: string (nullable = true)
 |-- NOME_FANTASIA: string (nullable = true)
 |-- CAP_SOCIAL: float (nullable = true)
 |-- NAT_JURICA: string (nullable = true)
 |-- NM_QUALIFICACAO: string (nullable = true)
 |-- SEXO_PROB: string (nullable = true)
 |-- NM_MATRIZ_FILIAL: string (nullable = true)
 |-- CNAE: string (nullable = true)
 |-- ano_cadastro: integer (nullable = true)
 |-- mes_cadastro: integer (nullable = true)
 |-- TIPO_LOUGRADOURO: string (nullable = true)
 |-- CEP: integer (nullable = true)
 |-- MUNICIPIO: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- PROVEDOR: string (nullable = true)
 |-- LON: float (nullable = true)
 |-- LAT: float (nullable = true)
 |-- DDD1: string (nullable = true)
 |-- TIPO_TELEFONE_FINAL: string (nullable = true)
 |-- OPERADORA: string (nullable = true)
 |-- renda_per_capita: float (nullable = true)
 |-- PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000: float (nullable = true)
 |-- P

## Correlação de Pearson

In [137]:
numeric_features = [t[0] for t in df.dtypes if t[1] in ('int', 'double', 'float', 'long')]
corr_df = pearson_correlation(df, numeric_features)

In [138]:
corr_df

,CAP_SOCIAL,ano_cadastro,mes_cadastro,CEP,renda_per_capita,PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,IDHM_2010
CAP_SOCIAL,1.000000,0.001417,0.000461,-0.000871,-0.000122,-0.000096,-0.000490,-0.000919
ano_cadastro,0.001417,1.000000,-0.050572,0.000367,0.014255,-0.012606,-0.006076,0.046624
mes_cadastro,0.000461,-0.050572,1.000000,-0.008125,0.000996,0.014658,0.022999,0.008694
CEP,-0.000871,0.000367,-0.008125,1.000000,-0.071294,-0.395151,-0.146983,-0.261718
renda_per_capita,-0.000122,0.014255,0.000996,-0.071294,1.000000,0.217990,0.168738,0.404031
PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000,-0.000096,-0.012606,0.014658,-0.395151,0.217990,1.000000,0.330399,0.378237
PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100,-0.000490,-0.006076,0.022999,-0.146983,0.168738,0.330399,1.000000,0.353569
IDHM_2010,-0.000919,0.046624,0.008694,-0.261718,0.404031,0.378237,0.353569,1.000000


## Teste de Chi-quadrado 

In [67]:
# Chi-Square Test
response_col = 'CAP_SOCIAL'
categorical_cols = ["NAT_JURICA", "NM_QUALIFICACAO", "SEXO_PROB", "NM_MATRIZ_FILIAL", "CNAE", "TIPO_LOUGRADOURO", "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF"]
chi_square_cols = chi_square_test(df, categorical_cols, response_col)

ValueError: list.remove(x): x not in list

In [ ]:
chi_square_cols

## Variance Threshold Selector

In [ ]:
# Variance Threshold Selector
numeric_features = ["ano_cadastro", "mes_cadastro", "CEP", "LON", "LAT", "renda_per_capita", "PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000", "PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100"]]
selected_features_variance = variance_threshold_selector(df, numeric_features, experiment_id)
selected_features_variance

## Univariate Feature Selector

In [ ]:
# Univariate Feature Selector
response_col = 'CAP_SOCIAL'
categorical_cols = ["NAT_JURICA", "NM_QUALIFICACAO", "SEXO_PROB", "NM_MATRIZ_FILIAL", "CNAE", "TIPO_LOUGRADOURO", "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF"]
numeric_features = ["ano_cadastro", "mes_cadastro", "CEP", "LON", "LAT", "renda_per_capita", "PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000", "PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100"]]
selected_features_univariate = univariate_feature_selector(df, categorical_cols, numeric_features, response_col)
selected_features_univariate

## RFormula

In [ ]:
# RFormula
categorical_cols = ["NAT_JURICA", "NM_QUALIFICACAO", "SEXO_PROB", "NM_MATRIZ_FILIAL", "CNAE", "TIPO_LOUGRADOURO", "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF"]
numeric_features = ["ano_cadastro", "mes_cadastro", "CEP", "LON", "LAT", "renda_per_capita", "PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000", "PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100"]
response_col = 'CAP_SOCIAL'
r_formula_df = r_formula(df, categorical_cols, numeric_features, response_col)
r_formula_df

## VectorSlicer

In [ ]:
# VectorSlicer
vector_slicer_function(df)
vector_slicer_function

In [5]:
# Primeiro, defina as colunas categóricas e numéricas
categorical_cols = ["NAT_JURICA", "NM_QUALIFICACAO", "SEXO_PROB", "NM_MATRIZ_FILIAL", "CNAE", "TIPO_LOUGRADOURO", "PROVEDOR", "DDD1", "TIPO_TELEFONE_FINAL", "OPERADORA", "MUNICIPIO", "UF"]
numeric_features = ["ano_cadastro", "mes_cadastro", "CEP", "LON", "LAT", "renda_per_capita", "PRODUTO_INTERNO_BRUTO_A_PREOS_CORRENTES_R_1000", "PRODUTO_INTERNO_BRUTO_PER_CAPITA_A_PREOS_CORRENTES_R_100"]

# 1. Realizando o teste Chi-quadrado
significant_columns_chi_square = chi_square_test(df, categorical_cols, "CAP_SOCIAL")

# 2. Avaliando a importância das features com Random Forest
significant_columns_rf = feature_importance_rf(df, categorical_cols, numeric_features, "CAP_SOCIAL")

# 3. Selecionando features com Chi-Square Selector
selected_features_chi_sq = chi_sq_selector(df, categorical_cols, "CAP_SOCIAL")

# 4. Usando Variance Threshold Selector
selected_features_variance = variance_threshold_selector(df, numeric_features)

# 5. Usando Univariate Feature Selector
selected_features_univariate = univariate_feature_selector(df, categorical_cols, numeric_features, "CAP_SOCIAL")
